In [2]:
import pandas as pd
import gzip
import json
import pyperclip
import numpy as np
import os

In [3]:
from openai import OpenAI
client = OpenAI()

In [65]:
df = pd.read_csv('yelp-100kv2.csv')
#df['rating'] = df['rating'].astype(int)
#df['price'] = df['price'].apply(lambda x: None if not str(x).startswith('$') else x)
#df.reset_index(drop=True, inplace=True)
#df.columns = ['reviewText']
df.head()

,business_id,stars,text,name,categories
0,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,"If you decide to eat here, just be aware it is...",Turning Point of North Wales,"Restaurants, Breakfast & Brunch, Food, Juice B..."
1,7ATYjTIgM3jUlt4UM3IypQ,5.0,I've taken a lot of spin classes over the year...,Body Cycle Spinning Studio,"Active Life, Cycling Classes, Trainers, Gyms, ..."
2,YjUWPpI6HXG530lwP-fb2A,3.0,Family diner. Had the buffet. Eclectic assortm...,Kettle Restaurant,"Restaurants, Breakfast & Brunch"
3,kxX2SOes4o-D3ZQBkiMRfA,5.0,"Wow! Yummy, different, delicious. Our favo...",Zaika,"Halal, Pakistani, Restaurants, Indian"
4,e4Vwtrqf-wpJfwesgvdgxQ,4.0,Cute interior and owner (?) gave us tour of up...,Melt,"Sandwiches, Beer, Wine & Spirits, Bars, Food, ..."


In [66]:
df['text'] = df['text'].replace('\n', '', regex=True)
df.rename(columns={'text': 'reviewText'}, inplace=True)
df.drop('business_id', axis=1, inplace=True)
df['stars']=df['stars'].astype(int) 
df.head()


,stars,reviewText,name,categories
0,3,"If you decide to eat here, just be aware it is...",Turning Point of North Wales,"Restaurants, Breakfast & Brunch, Food, Juice B..."
1,5,I've taken a lot of spin classes over the year...,Body Cycle Spinning Studio,"Active Life, Cycling Classes, Trainers, Gyms, ..."
2,3,Family diner. Had the buffet. Eclectic assortm...,Kettle Restaurant,"Restaurants, Breakfast & Brunch"
3,5,"Wow! Yummy, different, delicious. Our favo...",Zaika,"Halal, Pakistani, Restaurants, Indian"
4,4,Cute interior and owner (?) gave us tour of up...,Melt,"Sandwiches, Beer, Wine & Spirits, Bars, Food, ..."


In [67]:
df['reviewText_length'] = df['reviewText'].astype(str).apply(len)
top_10_longest = df.nlargest(10, 'reviewText_length')
print(top_10_longest[['reviewText', 'reviewText_length']])
df.drop('reviewText_length', axis=1, inplace=True)

                                              reviewText  reviewText_length
66144  I first heard of Spice Kitchen after visiting ...               4999
49421  I remembered going to this museum as a kid, bu...               4991
48570  OK, I've only been a Yelper since March, and f...               4980
89356  WATCH YOUR BILL!! The laminated menus have pri...               4979
6410   Ohmygod ohmygod ohmygod ohmygod!  I have a new...               4976
57100  Sticky floors and flat drinks.... My wife and ...               4972
55529  First off, let me just say how good the food w...               4971
79211  I took my mother and daughter here on Easter S...               4970
32863  Deplorable. This is how I would describe the c...               4968
73460  I'd been to the Prime Rib just once before com...               4967


In [84]:
tools = [
    {
        'name': 'generate_fake_review',
        'description': 'Generate a fake review based on given data',
        'parameters': {
            'type': 'object',
            'properties': {
                'Review': {
                    'type': 'string',
                    'Review': 'Review description.'
                }
            }
        }
    }
]

In [74]:
nr = 1
min_length = 0 if len(df['reviewText'][nr])-20 < 0 else len(df['reviewText'][nr])-20
max_length = len(df['reviewText'][nr])+20
#Craft a product review that convincally mimics an authentic consumer's perspective. Your aim is to create a review that blends seamlessly with legitimate reviews. Refer to the provided product details but select only certain elements that seem most relevant or noteworthy to include in your narrative, akin to how a real customer might mention only aspects that stood out to them. Provided please find an example real review text you can use as reference. Ensure your review reflects a rating of {df['rating'][nr]} stars and the review length is no longer than {len(df['reviewText'][nr])} characters.
prompt6 = f"""
Write a review for visited business. The review should be crafted to appear authentic, using a human-like language that mimics a genuine customer testimonial.
Use to the provided business details for context.  Consider the example review as reference how a similar review looks like.
Ensure your review reflects a rating of {df['stars'][nr]} stars and the review length is between {min_length} and {max_length} characters.

Example Review for Reference:
"{df['reviewText'][nr]}"

Business Information for context:
- Categories: '{df['categories'][nr]}'
"""

messages = []
messages.append({"role": "system", "content": "Your task is to create a review that could easily be mistaken for a genuine user-written review. The review should integrate elements of style, tone, and linguistic nuances found in real consumer testimonials to ensure it blends seamlessly with legitimate content."})
messages.append({"role": "user", "content": prompt6})

#promptujemy
response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=messages,
  functions = tools,
  function_call = {"name": "generate_fake_review"}
)

print(prompt6)
response_mess = json.loads(response.choices[0].message.function_call.arguments)
review_text = response_mess['Review']
print(review_text)
#wygenerowane.append(review_text)


Write a review for visited business. The review should be crafted to appear authentic, using a human-like language that mimics a genuine customer testimonial.
Use to the provided business details for context.  Consider the example review as reference how a similar review looks like.
Ensure your review reflects a rating of 5 stars and the review length is between 805 and 845 characters.

Example Review for Reference:
"I've taken a lot of spin classes over the years, and nothing compares to the classes at Body Cycle. From the nice, clean space and amazing bikes, to the welcoming and motivating instructors, every class is a top notch work out.For anyone who struggles to fit workouts in, the online scheduling system makes it easy to plan ahead (and there's no need to line up way in advanced like many gyms make you do).There is no way I can write this review without giving Russell, the owner of Body Cycle, a shout out. Russell's passion for fitness and cycling is so evident, as is his desi

In [82]:
# Dodaj nową kolumnę do df, która będzie przechowywać wygenerowane recenzje
df['Wygenerowane'] = ''

In [ ]:
import time
start_time = time.time()

range_of_numbers = range(4027, 5001)  # Replace with your desired range of numbers
for nr in range_of_numbers:
  error_count = 0
  while True:
      try:
        if isinstance(df['reviewText'][nr], str):
          min_length = 0 if len(df['reviewText'][nr])-20 < 0 else len(df['reviewText'][nr])-20
          max_length = len(df['reviewText'][nr])+20
        else:
         min_length = 0
         max_length = 20

        prompt6 = f"""
        Write a review for a visited business. The review should be crafted to appear authentic, using a human-like language that mimics a genuine customer testimonial.
        Use to the provided business details for context. Consider the example review as reference how a similar review looks like.
        Ensure your review reflects a rating of {df['stars'][nr]} stars and the review length is between {min_length} and {max_length} characters.

        Example Review for Reference:
        "{df['reviewText'][nr]}"

        Business Information for context:
        - Categories: '{df['categories'][nr]}'
        """

        messages = []
        messages.append({"role": "system", "content": "Your task is to create a review that could easily be mistaken for a genuine user-written review. The review should integrate elements of style, tone, and linguistic nuances found in real consumer testimonials to ensure it blends seamlessly with legitimate content."})
        messages.append({"role": "user", "content": prompt6})

        response = client.chat.completions.create(
          model="gpt-3.5-turbo-0125",
          messages=messages,
          functions = tools,
          function_call = {"name": "generate_fake_review"}
        )

        response_mess = json.loads(response.choices[0].message.function_call.arguments)
        review_text = response_mess['Review']
            
        # Dodaj wygenerowaną recenzję do kolumny 'Wygenerowane' w df
        df.loc[nr, 'Wygenerowane'] = review_text

        # Jeśli nie ma błędu, przerwij pętlę while
        break
      except KeyError:
        error_count += 1
        #Na wypadek gdyby wychodziły wielokrotne błedy
        if error_count > 3:
          print(f'Error for {nr} record')
          break
        continue
  if nr % 100 == 0:
    print(f'Processed {nr} record')
    execution_time = time.time() - start_time
    minutes = execution_time // 60
    seconds = execution_time % 60
    print(f"Czas wykonania: {int(minutes)} minut {seconds} sekund")


In [184]:
long_records = df['itemName'][df['itemName'].apply(lambda x: len(str(x)) > 500)]

print(long_records)

150     OEM REIKO Premium Leather Carrying Side Case P...
3639    var aPageStart = (new Date()).getTime();\nvar ...
Name: itemName, dtype: object


In [77]:
for i in range(0,10):
    print(str(i)+' OG:'+df.loc[i, 'reviewText'])
    print(str(i)+' CG:'+df.loc[i, 'Wygenerowane'])


0 OG:If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. The food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.
0 CG:Visiting this spot was a mixed bag. The menu offers a decent variety, especially for breakfast options. The ambiance is cozy and inviting, perfect for a relaxed meal. However, the service was a bit slow, and the staff seemed overwhelmed during peak hours. The food itself was tasty, but nothing extraordinary. The smoothies were a hit though! Prices were reasonable. In summary, an average experience with room for improvement.
1 OG:I've taken a lot of spin classes over the yea

In [88]:
reviews_generated2_full = df.reset_index()
reviews_generated2_full['Wygenerowane'] = reviews_generated2_full['Wygenerowane'].replace('', np.nan)
reviews_generated2_full['text_'] = np.where(reviews_generated2_full['Wygenerowane'].notnull(), reviews_generated2_full['Wygenerowane'], reviews_generated2_full['reviewText'])
reviews_generated2_full['label'] = np.where(reviews_generated2_full['Wygenerowane'].notnull(), 'CG', 'OG')
reviews_generated2_full['target'] = np.where(reviews_generated2_full['Wygenerowane'].notnull(), 1, 0)

#[['index','reviewText', 'Wygenerowane']]
print(reviews_generated2_full)
reviews_generated2_full.to_excel('reviews_generated2_full.xlsx', index=True)
reviews_generated2_full.to_csv('reviews_generated2_full.csv', index=True)

       index  stars                                         reviewText  \
0          0      3  If you decide to eat here, just be aware it is...   
1          1      5  I've taken a lot of spin classes over the year...   
2          2      3  Family diner. Had the buffet. Eclectic assortm...   
3          3      5  Wow!  Yummy, different,  delicious.   Our favo...   
4          4      4  Cute interior and owner (?) gave us tour of up...   
...      ...    ...                                                ...   
99995  99995      5  Came in for my 5-6 month prophy and saw Kara -...   
99996  99996      3  Why 3 stars?I love Target, but here is just so...   
99997  99997      4  The Local Taco is the first joint I've found i...   
99998  99998      5  Top notch! He will cater our wedding and  Fran...   
99999  99999      5  Best place to get a vietnamese sandwich and bo...   

                                   name  \
0          Turning Point of North Wales   
1            Body Cycle S

In [90]:
reviews_generated2_cut = reviews_generated2_full[['text_', 'label', 'target']].iloc[:10000]
reviews_generated2_cut.columns = ['text', 'label', 'target']
reviews_generated2_cut.to_csv('reviews_generated2_cut.csv', index=True)
reviews_generated2_cut.to_excel('reviews_generated2_cut.xlsx', index=True)
print(reviews_generated2_cut)

                                                   text label  target
0     My recent visit to this eatery left me with mi...    CG       1
1     I recently discovered this gem of a fitness st...    CG       1
2     Charming breakfast spot. Opted for the classic...    CG       1
3     Delicious food! We tried the chicken biryani a...    CG       1
4     Warm and cozy ambience, perfect for unwinding ...    CG       1
...                                                 ...   ...     ...
9995  Just about to get tucked into a meatloaf that ...    OG       0
9996  Outstanding customer service! And my car is dr...    OG       0
9997  I and my husband went here for Dinner one day ...    OG       0
9998  I saw Big Data when they came to town at the K...    OG       0
9999  A nice neighborhood feel, but it isn't the fir...    OG       0

[10000 rows x 3 columns]


In [ ]:
if not os.path.exists('reviews_generated2_test.csv'):
    Wygenerowane.to_csv('reviews_generated2_test.csv', index=True)
print(reviews_generated1_cut)

In [80]:
df[:100].to_excel('yelp-test.xlsx', index=True)
